## YΣ19 Artificial Intelligence II
# Homework 2

### Iglezou Myrto - 111520170038

# Model 2

In [ ]:
import pandas as pd 
from pandas import DataFrame, read_csv

In [ ]:
# import gdown
# url = 'https://drive.google.com/file/d/1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79/view?usp=sharing'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# output = "twitterData.csv"
# gdown.download(path, output, quiet=False)
# twitterData = pd.read_csv("/content/twitterData.csv")
# df = pd.DataFrame(data=twitterData)
# df.head(5)

path = r"C:\Users\myrto\Documents\GitHub\AI2-project2\SentimentTweets.csv"
file = pd.read_csv(path,index_col=False,nrows=50000)
df = pd.DataFrame(data=file)
df.head(5)

In [ ]:
df['target'].replace({4: 1}, inplace = True)
X = df['text']
y = df['target']

## Data pre-processing

### Removing the scpecial characters

In [ ]:
import re

def RemoveSpecialCharacters(x):
    x = re.sub(r'[^\w]+', " ", x)
    return x

In [ ]:
X = X.apply(lambda x: RemoveSpecialCharacters(x) )

### Lemmatize

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

X = X.apply(lambda x: WordNetLemmatizer().lemmatize(x))

In [ ]:
X = X.apply(lambda x: x.lower())

### Removing the upercase

In [ ]:
X = X.apply(lambda x: x.lower())